Download the text--Moby Dick, remove the gutenberg_ig column and remove rows that are empty

In [2]:
import re
#from functools import reduce
#import rpy2
from rpy2.robjects.packages import importr
import rpy2.robjects as ro
from rpy2.robjects import pandas2ri

pandas2ri.activate()
importr('gutenbergr')

df = ro.r('gutenberg_download("2701")')
df.drop('gutenberg_id', inplace=True, axis=1)
df = df[df['text'] != ''].reset_index(drop=True)
#df.drop('index', inplace=True, axis=1)

In [3]:
df.iloc[:15]['text']

0                               MOBY DICK; OR THE WHALE
1                                    By Herman Melville
2                         Original Transcriber's Notes:
3     This text is a combination of etexts, one from...
4     project at Virginia Tech and one from Project ...
5     proofreaders of this version are indebted to T...
6     Library for preserving the Virginia Tech versi...
7     was compared with a public domain hard copy ve...
8     In chapters 24, 89, and 90, we substituted a c...
9            for the British pound, a unit of currency.
10                                           ETYMOLOGY.
11    (Supplied by a Late Consumptive Usher to a Gra...
12    The pale Usher--threadbare in coat, heart, bod...
13    now. He was ever dusting his old lexicons and ...
14    handkerchief, mockingly embellished with all t...
Name: text, dtype: object

Capture the title

In [4]:
title = df.iloc[0]['text']
title

'MOBY DICK; OR THE WHALE'

Capture the author

In [5]:
author = df.iloc[1]['text']
author = re.sub(r'^By ','',author)
author

'Herman Melville'

Remove the text up until Chapter 1 begins

In [6]:
df.drop(df.index[0:df[df.text.str.contains('^CHAPTER 1\.')].index.tolist()[0]], inplace=True)
df.reset_index(inplace=True, drop=True)

Make a table of contents list with chapters and epilogue

In [7]:
TOC = df[df.text.str.contains('^CHAPTER|Epilogue')]['text'].tolist()

In [8]:
len(TOC)

136

Combine chapter data together

In [9]:
corpus_dict = {}
i = 0
for k,v in df[1:].to_dict()['text'].items():
    if i not in corpus_dict:
        corpus_dict[i] = []
    if v not in TOC:
        corpus_dict[i].append(v)
    else:
        i += 1
        corpus_dict[i] = []

In [10]:
len(corpus_dict)

136

Parse text into sentences, store them in a dict as prepartion to move into a dataframe with columns of title, chapter and sentence.

In [11]:
from nltk.tokenize import sent_tokenize
sentence_dict = {}
sentence_dict['title'] = []
sentence_dict['chapter'] = []
sentence_dict['sentence'] = []
sentence_dict['author'] = []
for k,v in corpus_dict.items():
    for sentence in sent_tokenize(' '.join(v)):
        sentence_dict['title'].append(title)
        sentence_dict['chapter'].append(TOC[k])
        sentence_dict['sentence'].append(sentence)
        sentence_dict['author'].append(author)

Place the dict into a data frame so the data can be output to a csv easily.

In [12]:
import pandas as pd
final_df = pd.DataFrame.from_dict(sentence_dict)

In [13]:
final_df.sample(5)

,author,chapter,sentence,title
3268,Herman Melville,CHAPTER 41. Moby Dick.,But that thing of his dissembling was only sub...,MOBY DICK; OR THE WHALE
8905,Herman Melville,CHAPTER 133. The Chase--First Day.,The bluish pearl-white of the inside of the ja...,MOBY DICK; OR THE WHALE
8611,Herman Melville,CHAPTER 129. The Cabin.,And now I quit thee.,MOBY DICK; OR THE WHALE
7455,Herman Melville,CHAPTER 108. Ahab and the Carpenter.,"No, but put a sky-light on top of his head to ...",MOBY DICK; OR THE WHALE
8116,Herman Melville,CHAPTER 121. Midnight.--The Forecastle Bulwarks.,"any man with half an eye can be sensible.""",MOBY DICK; OR THE WHALE


In [14]:
final_df.to_csv('./moby_dick.csv', index=False)